In [355]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np


In [356]:
# Load the Titanic dataset (replace with your dataset path)
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')



In [357]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [358]:
# Data Preprocessing
def preprocess_data(df):
    # Fill missing values
    df['Age'].fillna( df['Age'].mean(), inplace=True)
    df['Embarked'].fillna( df['Embarked'].mode()[0], inplace=True)
    df['Fare'].fillna( df['Fare'].mean(), inplace=True)
    
    
    df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
    
    # Encode categorical features
    le = LabelEncoder()
    df['Sex'] = le.fit_transform(df['Sex'])
    df['Embarked'] = le.fit_transform(df['Embarked'])
    
    return df

train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)

# Split the data into features and labels
X = train_df.drop('Survived', axis=1).values
y = train_df['Survived'].values


In [359]:
# Preprocess the training data
#X_train, y_train = preprocess_data(train_data)
# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train
 # Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)


In [360]:
X_train_tensor.shape

torch.Size([712, 8])

In [361]:
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [362]:
# Define the Neural Network with Dropout
class TitanicMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(TitanicMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        return x
# Instantiate the model
input_size = X_train.shape[1] #Number of features
hidden_size = 64
output_size = 2  # Binary classification (Survived or not)
dropout_rate = 0.5  # Adjust as needed
model = TitanicMLP(input_size, hidden_size, output_size, dropout_rate)


In [363]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [364]:
# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in train_dataloader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        
    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in val_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            val_loss += criterion(outputs, labels)

        avg_val_loss = val_loss / len(val_dataloader)
        accuracy = correct / total * 100.0

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%')

Epoch [1/10], Loss: 15.0918, Validation Loss: 6.5101, Accuracy: 57.54%
Epoch [2/10], Loss: 10.8215, Validation Loss: 3.2232, Accuracy: 59.22%
Epoch [3/10], Loss: 9.9793, Validation Loss: 1.9504, Accuracy: 60.34%
Epoch [4/10], Loss: 12.0388, Validation Loss: 1.2154, Accuracy: 62.01%
Epoch [5/10], Loss: 3.1293, Validation Loss: 2.3871, Accuracy: 62.01%
Epoch [6/10], Loss: 9.8914, Validation Loss: 1.7411, Accuracy: 62.01%
Epoch [7/10], Loss: 6.4962, Validation Loss: 1.9207, Accuracy: 62.57%
Epoch [8/10], Loss: 3.3613, Validation Loss: 0.7301, Accuracy: 71.51%
Epoch [9/10], Loss: 4.2709, Validation Loss: 1.8052, Accuracy: 63.13%
Epoch [10/10], Loss: 0.9076, Validation Loss: 0.6074, Accuracy: 70.39%


In [365]:
# Prediction on test set
X_test_tensor = torch.tensor(test_df.values, dtype=torch.float32)
test_dataset = TensorDataset(X_test_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()
predictions = []
with torch.no_grad():
    for inputs in test_dataloader:
        outputs = model(inputs[0])
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.numpy())

        print("Predicted Labels:")
print(predicted_labels)

In [366]:
# Save predictions to a CSV file
submission_df = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': predictions})
submission_df.to_csv('/kaggle/working/titanic_predictions.csv', index=False)